<a href="https://colab.research.google.com/github/UniVR-DH/DBMS-course/blob/main/notebooks/lab03-sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL exercise with SQLite in Jupyter Notebooks
In this notebook we use SQLite as a DBMS, plus we use some plugins to simplify your way to run SQL queries.

## Library Import and Configuration

In [1]:
!pip install --quiet jupysql
!pip install --quiet pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.8 MB/s eta 0:00:00


In [2]:
import sqlite3
import pandas as pd
# Import jupysql Jupyter extension to create SQL cells
# this avoids the need to run SQL in python
%load_ext sql

**We configure jupysql to return data as a Pandas dataframe and have less verbose output**

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Initialize the Database

In [4]:
# Run SQLite saving data to a file called `mydb.db` -- you can change the name
%sql sqlite:///mydb.db

An entire Jupyter cell can be used as a SQL cell by placing `%%sql` at the start of the cell. Query results will be displayed as a Pandas DF.

In [5]:
%%sql
SELECT 1=2 as test, 'Hello people' as message, 3*12345 as math  ;

,test,message,math
0,0,Hello people,37035


## Create tables with some data

In [6]:
%%sql
CREATE TABLE customer (
    id INTEGER PRIMARY KEY,
    name varchar(25)
);


CREATE TABLE c_order (
    ordid INTEGER PRIMARY KEY,
    date DATE,
    cid INTEGER DEFAULT 0,
    FOREIGN KEY (cid) REFERENCES customer(id)
);


CREATE TABLE invoice (
    ordid INTEGER,
    price INTEGER,
    cid INTEGER DEFAULT 0,
    part INTEGER DEFAULT 0,
    FOREIGN KEY (ordid) REFERENCES customer_order(ordid),
    FOREIGN KEY (cid) REFERENCES customer(id)
    PRIMARY KEY (part, ordid)
);

""


In [7]:
%%sql
INSERT INTO customer (id, name)
VALUES
(2101, "Christian"),
(2102, "Jens"),
(2103, "Kristian"),
(2104, "Christian");


INSERT INTO c_order (ordid, date, cid)
VALUES
(97001, '2024-10-1', 2101),
(97002, '2024-06-1', 2101),
(97003, '2023-11-1', 2103),
(97004, '2023-11-1', 0)
;


INSERT INTO invoice (ordid, price, cid, part)
VALUES
(97001, 100, 2101, 1),
(97001, 100, 2101, 2),
(97003, 90, 2103, 1),
(10004, 10, 0, 1),
(10005, 10, 2103, 1)
;

""


## Ex1: For all invoices if they have an order print the date, if they have a customer print the name


In [8]:
%%sql
SELECT i.ordid, o.date, c.name
FROM invoice i
LEFT JOIN c_order o ON i.ordid = o.ordid
LEFT JOIN customer c ON i.cid = c.id

,ordid,date,name
0,97001,2024-10-1,Christian
1,97001,2024-10-1,Christian
2,97003,2023-11-1,Kristian
3,10004,None,None
4,10005,None,Kristian


## Ex2: Find invoices without orders and customers

In [9]:
%%sql
SELECT *
FROM invoice i
LEFT JOIN c_order o ON i.ordid = o.ordid
LEFT JOIN customer c ON i.cid = c.id
WHERE o.ordid IS NULL AND c.id IS NULL

,ordid,price,cid,part,ordid,date,cid,id,name
0,10004,10,0,1,None,None,None,None,None


In [10]:
%%sql
SELECT *
FROM invoice i
WHERE i.ordid NOT IN  (SELECT ordid FROM c_order)
     AND i.cid NOT IN (SELECT id FROM customer)

,ordid,price,cid,part
0,10004,10,0,1


## Ex3: Find the total amount owed by each customer


In [11]:
%%sql
SELECT c.id, c.name, SUM(i.price) AS total_owed
FROM customer c
JOIN invoice i ON c.id = i.cid
GROUP BY c.id, c.name;

,id,name,total_owed
0,2101,Christian,200
1,2103,Kristian,100


## Ex4: Find the total amount owed by each customer inlcuding unknown




In [12]:
%%sql
SELECT
  c.name,
  SUM(i.price) AS total_owed
FROM invoice AS i
LEFT JOIN customer AS c ON i.cid = c.id
GROUP BY c.name;

,name,total_owed
0,None,10
1,Christian,200
2,Kristian,100


## Ex5: Find the total amount owed by each customer inlcuding unknown, grouped by order id